## Week 3 Assignment

In this assignment I will be exploring an clustering neighbourhoods in Toronto.

### Part 1

Import neighbourhood data from Wikipedia and create a dataframe of postal code, borough and neighbourhood.

In [73]:
#Import required libraries
!pip install bs4
!pip install pandas
!pip install sklearn
!pip install matplotlib
import requests
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup

     |████████████████████████████████| 10.3 MB 18.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 67.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 53.2 MB/s eta 0:00:01


In [3]:
#Create soup object for Wikipedia page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result=requests.get(url).content
soup=BeautifulSoup(result)
#soup

In [4]:
#Extract relevant data and convert to text string.
#Format string in suitable format to create a list. 
table=soup.find('table')
Text=table.text
Text
Text1=Text.replace('\n','_').replace('____','_').replace('___','_').replace('__','_').replace('(',',').replace(')','').replace('Not assigned','Not assigned, ')
Text1

#Create list where each element is one post code.
List=list(Text1.split("_"))
del(List[0])
del(List[-1])
#List

In [5]:
#Loop through List to add a space after each post code.
i=0

while i < len(List):
    a=List[i]
    b=a[0:3] + ',' + a[3:]
    List[i]=b  
    i=i+1

#List

In [6]:
#Split List into 3 lists for Postcode, neighbourhood and boroughs
list1, list2, list3 = [x.split(',')[0] for x in List], [x.split(',')[1] for x in List] , [x.split(',')[2] for x in List]
print('PostalCodes: ',len(list1))
print('Boroughs: ',len(list2))
print('Neighbourhoods: ',len(list3))

PostalCodes:  180
Boroughs:  180
Neighbourhoods:  180


In [7]:
#Create Pandas dataframe.
df = pd.DataFrame(list(zip(list1, list2, list3)),
               columns =['PostalCode', 'Borough', 'Neighbourhood'])

df['Neighbourhood']=df['Neighbourhood'].str.replace(' / ',', ')
df.shape

(180, 3)

In [8]:
#Drop rows with and not Assigned Boroughs.
df=df[df['Borough'] != 'Not assigned' ]
df.reset_index(drop=True, inplace=True)
df.shape

(103, 3)

In [9]:
#Check no 'Not Assigned' or missing Neighbourhoods
NANeighbourhood=df[df['Neighbourhood'] == 'Not assigned']
MissNeighbourhood=df[df['Neighbourhood'] == '']
NANeighbourhood 
MissNeighbourhood
#No observations

,PostalCode,Borough,Neighbourhood


In [10]:
#Check no duplicated postal codes.
dupPC = df[df['PostalCode'].duplicated()]
dupPC
#No observations

,PostalCode,Borough,Neighbourhood


In [11]:
#Demonstrate number of rows and columns in dataframe
df.shape

(103, 3)

### Part 2

Add latitude and longitude coordinates for each Postal Code

In [12]:
#Get latitude and longitude coordinates for each post code.
        #*****Note: Unable to get geocoder to work - using csv file instead*****

#!pip install geocoder
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

#postal_code=df['PostalCode']

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]


#Read in csv file, convert to dataframe and adjust as required for merge
csv='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
df_csv = pd.read_csv(csv)
df_csv.rename({'Postal Code': 'PostalCode'}, axis=1, inplace=True) #Rename as per df dataframe.
df_csv

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [13]:
#Merge original dataframe and csv dataframe to map lat and long to original data.
df_out=pd.merge(
    df,
    df_csv,
    how="inner",
    on='PostalCode',
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
#    suffixes=None,
    copy=True,
    indicator=False,
    validate=None,
)
df_out.shape

(103, 5)

In [14]:
#Check for missing co-ordinates
df_out.info()

#No missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PostalCode     103 non-null    object 
 1   Borough        103 non-null    object 
 2   Neighbourhood  103 non-null    object 
 3   Latitude       103 non-null    float64
 4   Longitude      103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


In [15]:
#Demonstrate final dataframe
df_out

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


### Part 3

In Part 3 I will explore and cluster neighbourhoods in Toronto. I will start by visualising the data by plotting each neighbourhood on a map. I will then choose a specific area(s) to focus on.

In [69]:
#Install libraries and assign Toronto coordinates
!pip install folium
import folium 

Tor_lat=43.6532
Tor_long=-79.3832


In [17]:
Tor_map=folium.Map(location=[Tor_lat, Tor_long],zoom_start=11)

for lat,long,pc,bor in zip(df_out['Latitude'], df_out['Longitude'], df_out['PostalCode'], df_out['Borough']):
    label=pc+' '+bor
    folium.CircleMarker(location=[lat, long],radius=2,weight=5).add_child(folium.Popup(label)).add_to(Tor_map)
    
Tor_map

In [18]:
#Check number of post codes for each borough 
df_out['Borough'].value_counts()

North York                                                      24
Downtown Toronto                                                17
Scarborough                                                     17
Etobicoke                                                       11
Central Toronto                                                  9
West Toronto                                                     6
York                                                             5
East Toronto                                                     4
East York                                                        4
MississaugaCanada Post Gateway Processing Centre                 1
EtobicokeNorthwest                                               1
Queen's Park                                                     1
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East YorkEast Toronto                                         

#### Note:
I will base the rest of the neighbourhood clustering/exploring analysis on the following four boroughs: 
Downtown Toronto, Central Toronto, West Toronto, and East Toronto. 

I will also make some assumptions and include 'East TorontoBusiness reply mail Processing Centre969 Eastern' in the 'East Toronto' borough, and the 'Downtown TorontoStn A PO Boxes25 The Esplanade' in the 'Downtown Toronto' borough.

In [19]:
#Change Borough values as described above and confirm that the change was effective
df_out["Borough"].replace({"East TorontoBusiness reply mail Processing Centre969 Eastern": "East Toronto", "Downtown TorontoStn A PO Boxes25 The Esplanade": "Downtown Toronto"}, inplace=True)
df_out['Borough'].value_counts()

North York                                          24
Downtown Toronto                                    18
Scarborough                                         17
Etobicoke                                           11
Central Toronto                                      9
West Toronto                                         6
York                                                 5
East Toronto                                         5
East York                                            4
MississaugaCanada Post Gateway Processing Centre     1
EtobicokeNorthwest                                   1
Queen's Park                                         1
East YorkEast Toronto                                1
Name: Borough, dtype: int64

In [20]:
#Subset dataframe to extract specific boroughs noted above
df_sub=df_out.loc[df_out['Borough'].isin(['Downtown Toronto','Central Toronto','West Toronto','East Toronto'])].reset_index(drop=True)
df_sub['Borough'].value_counts()

Downtown Toronto    18
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [21]:
#Re-plot neighbourhoods from subsetted dataframe
sub_map=folium.Map(location=[Tor_lat, Tor_long],zoom_start=12)

for lat,long,pc,bor in zip(df_sub['Latitude'], df_sub['Longitude'], df_sub['PostalCode'], df_sub['Borough']):
    label=pc+' '+bor
    folium.CircleMarker(location=[lat, long],radius=2,weight=5).add_child(folium.Popup(label)).add_to(sub_map)
    
sub_map

In the following steps I will call the FourSquare API to gather information about venues in proximity to each of the above neighbourhoods

In [22]:
CLIENT_ID = 'H41B2EZTCCABXAN4W2VQMMEXVWOF2GYP2MZ3QLLRVSLIQSQF' # your Foursquare ID
CLIENT_SECRET = 'YOZHRR2KQROR0JT5PSSGPMNHIRSQDU2HEYB5UWUOPP1ODRTE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        API_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(API_url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# type your answer here

toronto_venues=getNearbyVenues(df_sub['Neighbourhood'], df_sub['Latitude'], df_sub['Longitude'])
toronto_venues

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Roncesvalles

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
...,...,...,...,...,...,...,...
1588,Enclave of M4L,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
1589,Enclave of M4L,43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
1590,Enclave of M4L,43.662744,-79.321558,Greenwood Cigar & Variety,43.664538,-79.325379,Smoke Shop
1591,Enclave of M4L,43.662744,-79.321558,ONE Academy,43.662253,-79.326911,Gym / Fitness Center


In [23]:
df_to_merge=df_sub[['Neighbourhood','Borough']].copy()
df_to_merge

toronto_venues=pd.merge(
    toronto_venues,
    df_to_merge,
    how='left',
    on='Neighbourhood',
    left_index=False,
    right_index=False,
    sort=True,
    copy=True,
    indicator=False,
    validate=None,
)
toronto_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough
0,Berczy Park,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store,Downtown Toronto
1,Berczy Park,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant,Downtown Toronto
2,Berczy Park,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant,Downtown Toronto
3,Berczy Park,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum,Downtown Toronto
4,Berczy Park,43.644771,-79.373306,Goose Island Brewhouse,43.647329,-79.373541,Beer Bar,Downtown Toronto
...,...,...,...,...,...,...,...,...
1588,"University of Toronto, Harbord",43.662696,-79.400049,Magic Noodle,43.662728,-79.403602,Noodle House,Downtown Toronto
1589,"University of Toronto, Harbord",43.662696,-79.400049,Smoke's Poutinerie/Weinerie,43.658259,-79.400136,Poutine Place,Downtown Toronto
1590,"University of Toronto, Harbord",43.662696,-79.400049,The Beer Store,43.665385,-79.403477,Beer Store,Downtown Toronto
1591,"University of Toronto, Harbord",43.662696,-79.400049,Caversham Booksellers,43.662951,-79.403418,Bookstore,Downtown Toronto


In [24]:
#How many venues are in each Neighbourhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough
Neighbourhood,,,,,,,
Berczy Park,59,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23,23
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16,16
Central Bay Street,63,63,63,63,63,63,63
Christie,16,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100,100
Davisville,34,34,34,34,34,34,34
Davisville North,8,8,8,8,8,8,8


In [25]:
toronto_venues.groupby('Venue Category').count()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Borough
Venue Category,,,,,,,
Adult Boutique,1,1,1,1,1,1,1
Airport,1,1,1,1,1,1,1
Airport Food Court,1,1,1,1,1,1,1
Airport Gate,1,1,1,1,1,1,1
Airport Lounge,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...
Vegetarian / Vegan Restaurant,16,16,16,16,16,16,16
Video Game Store,2,2,2,2,2,2,2
Vietnamese Restaurant,7,7,7,7,7,7,7


##### Next, I will summarise the venues data and analyse the top venue types for each Neighbourhood

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
#Group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.016949,0.00000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.043478
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.015873,0.00000,0.000000,0.015873,0.015873
4,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.012821,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012821,0.000000,...,0.012821,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.025641
6,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.020000,0.00000,0.000000,0.010000,0.000000
7,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.029412,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000


#### Create data frame of Top 10 venues for each Neighbourhood

In [28]:
#Create a function to sort data in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
#Create dataframe of top 10 venues for each Neighbourhoo
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Farmers Market,Seafood Restaurant,Cheese Shop,Beer Bar,Restaurant,Steakhouse
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Italian Restaurant,Burrito Place,Stadium,Furniture / Home Store,Bar,Climbing Gym
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Rental Car Location,Airport Food Court,Airport Gate,Sculpture Garden,Harbor / Marina,Airport
3,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Bubble Tea Shop,Salad Place,Burger Joint,Diner,Dessert Shop
4,Christie,Grocery Store,Café,Park,Coffee Shop,Restaurant,Italian Restaurant,Baby Store,Nightclub,Athletics & Sports,Candy Store


#### Next: 
   Visualise the top venue in each neighbourhood on a map - colour co-ordinating by venue type to identify if there is commonality in the top venues by Borough

In [30]:
#Create list of top venues across neighbourhoods (use set function to remove duplicates)
top_venue_list=list(neighborhoods_venues_sorted['1st Most Common Venue'])
top_venue_list=set(top_venue_list)
top_venue_list

{'Airport Service',
 'Bar',
 'Breakfast Spot',
 'Café',
 'Clothing Store',
 'Coffee Shop',
 'Construction & Landscaping',
 'Dessert Shop',
 'Greek Restaurant',
 'Grocery Store',
 'Gym / Fitness Center',
 'Home Service',
 'Mexican Restaurant',
 'Neighborhood',
 'Park',
 'Pharmacy',
 'Pizza Place',
 'Playground',
 'Sandwich Place'}

In [35]:
#Create colour list and then create dictionary from colours and venues
colour_list=['red', 
             'blue', 
             'green', 
             'purple', 
             'orange', 
             'darkred', 
             'beige', 
             'darkblue', 
             'darkgreen', 
             'cadetblue', 
             'darkpurple', 
             'white', 
             'pink', 
             'lightblue', 
             'lightgreen', 
             'gray', 
             'black',
            'lightred',
            'lightgray']

venue_colour_dict=dict(zip(top_venue_list,colour_list))
print(venue_colour_dict)

{'Home Service': 'red', 'Pizza Place': 'blue', 'Mexican Restaurant': 'green', 'Coffee Shop': 'purple', 'Pharmacy': 'orange', 'Park': 'darkred', 'Airport Service': 'beige', 'Breakfast Spot': 'darkblue', 'Sandwich Place': 'darkgreen', 'Grocery Store': 'cadetblue', 'Dessert Shop': 'darkpurple', 'Neighborhood': 'white', 'Greek Restaurant': 'pink', 'Bar': 'lightblue', 'Gym / Fitness Center': 'lightgreen', 'Clothing Store': 'gray', 'Construction & Landscaping': 'black', 'Café': 'lightred', 'Playground': 'lightgray'}


In [49]:
#Plot neighbourhoods (top venue in each) colour-coded by venue type
top_venues_to_map=pd.merge(
    df_sub[['Neighbourhood','Latitude','Longitude','Borough']],
    neighborhoods_venues_sorted[['Neighbourhood','1st Most Common Venue']],
    how='inner',
    on='Neighbourhood',
    left_index=False,
    right_index=False,
    sort=True,
    copy=True,
    indicator=False,
    validate=None,
)

top_venues_to_map['colour'] = top_venues_to_map['1st Most Common Venue'].map(venue_colour_dict)
top_venues_to_map

venues_map=folium.Map(location=[Tor_lat,Tor_long],zoom_start=13)

for lat,long,bor,venue,colour in zip(top_venues_to_map['Latitude'],top_venues_to_map['Longitude'],
                                     top_venues_to_map['Borough'],top_venues_to_map['1st Most Common Venue'],
                                     top_venues_to_map['colour']):
    label=venue+' '+bor
    folium.CircleMarker(location=[lat, long],radius=2,weight=5,color=colour,zoom=15).add_child(folium.Popup(label)).add_to(venues_map)
    
venues_map

Now, for the final step, I will cluster neighbourhoods.

In [55]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 7, 1, 1, 1, 5, 1], dtype=int32)

In [67]:
# add clustering labels
toronto_data = df_sub.drop(['PostalCode'],axis=1)

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data

# merge neighborhoods_venues_sorted with manhattan_merged to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,4,Neighborhood,Health Food Store,Trail,Pub,Music Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Restaurant,Bubble Tea Shop,Caribbean Restaurant,Brewery,Yoga Studio
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Sandwich Place,Fast Food Restaurant,Playground,Italian Restaurant,Pizza Place,Pub,Coffee Shop,Restaurant,Movie Theater,Brewery
3,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Brewery,Gastropub,Café,American Restaurant,Bakery,Cheese Shop,Clothing Store,Comfort Food Restaurant,Convenience Store
4,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Construction & Landscaping,Park,Bus Line,Swim School,Museum,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


In [74]:
# create map
map_clusters = folium.Map(location=[Tor_lat, Tor_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We can see that from the above map, the majority of neighbourhoods in Toronto East, Toronto West, Downtown Toronto and North Toronto, are very similar (falling into cluster 1). The above map is very much aligned with the earlier map (venues_map) which was created based on the most popular venue alone.

### Conclusion: 
The neighbourhoods in Toronto East, Toronto West, Downtown Toronto and North Toronto, are very similar in terms of popular venues